<a href="https://colab.research.google.com/github/da-shinjk/dlworkshop_02/blob/master/dlworkshop_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

第1回で入力したコード。

この環境で動作しているPythonバージョンを調べます。

In [1]:
import sys
print(sys.version)

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


以下からが第2回の内容です。

まず、あやめの特徴を示したデータセットをロードします。

In [0]:
from sklearn.datasets import load_iris
iris = load_iris()

データセットの説明を表示します。

In [3]:
print(iris.DESCR)

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

あやめのデータセットに含まれるデータや属性を幾つか表示します。

In [4]:
print(iris.feature_names)  # iris.data属性の説明
print(iris.data[0:5])  # iris.data属性の先頭5要素を表示
print(iris.target_names)  # iris.target属性の説明
print(iris.target[0:5])  # iris.target属性の先頭5要素を表示

['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
['setosa' 'versicolor' 'virginica']
[0 0 0 0 0]


読み込んだデータを、訓練データとテストデータに分割します。

In [8]:
from sklearn.model_selection import train_test_split
print('length of iris.data:', len(iris.data))  # iris.dataのデータ数
print('length of iris.target:', len(iris.target))  # iris.targetのデータ数

# iris.dataとiris.targetに含まれるデータをシャッフルして分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target)
print('length of X_train:', len(X_train))
print('length of y_train:', len(y_train))
print('length of X_test:', len(X_test))
print('length of y_test:', len(y_test))

length of iris.data: 150
length of iris.target: 150
length of X_train: 112
length of y_train: 112
length of X_test: 38
length of y_test: 38


分割された訓練データ（と対応する正解ラベル）の先頭から5つの要素を表示してみます。

In [9]:
for idx, item in enumerate(zip(X_train, y_train)):
    if idx == 5:
        break
    print(f'{item[0]}: {iris.target_names[item[1]]}')

[5.4 3.  4.5 1.5]: versicolor
[6.5 3.  5.5 1.8]: virginica
[5.3 3.7 1.5 0.2]: setosa
[4.8 3.4 1.9 0.2]: setosa
[6.2 3.4 5.4 2.3]: virginica


In [0]:
import torch

X_train = torch.from_numpy(X_train).float()
y_train = torch.tensor([[float(x)] for x in y_train])
X_test = torch.from_numpy(X_test).float()
y_test = torch.tensor([[float(x)] for x in y_test])

PyTorchのテンソルの扱い方を簡単に見てみるコードです。

In [15]:
# X_trainのサイズを調べる
print(X_train.size())
print(X_train.shape)  # NumPyと同様な書き方
print(len(X_train))  # 最初の次元の要素数（行数）
print(X_train.dim())  # テンソルの次元数
print(X_train.ndim)  # NumPyと同様な書き方

# テンソルの要素のデータ型を調べる
print(X_train.dtype)
print(y_train.dtype)

# テンソルの要素へのアクセス
print(X_train[0])  # 先頭行
print(X_train[0][0])  # 先頭行の先頭要素
print(X_train[1:3])  # 第1行と第2行
print(X_train[1:3, 1:3])  # 第1行と第2行の1列目と2列目の要素
print(X_train[1:3][1:3])  # この書き方は意味が異なる点に注意

torch.Size([112, 4])
torch.Size([112, 4])
112
2
2
torch.float32
torch.float32
tensor([5.4000, 3.0000, 4.5000, 1.5000])
tensor(5.4000)
tensor([[6.5000, 3.0000, 5.5000, 1.8000],
        [5.3000, 3.7000, 1.5000, 0.2000]])
tensor([[3.0000, 5.5000],
        [3.7000, 1.5000]])
tensor([[5.3000, 3.7000, 1.5000, 0.2000]])


ニューラルネットワークを定義するクラス。

In [0]:
from torch import nn

INPUT_FEATURES = 4
HIDDEN = 5
OUTPUT_FEATURES = 1

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(INPUT_FEATURES, HIDDEN)
        self.fc2 = nn.Linear(HIDDEN, OUTPUT_FEATURES)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.sigmoid(x)
        x = self.fc2(x)
        return x

torch.nn.Linearクラスのインスタンスが\__call__メソッドを持つかを確認します。

持っていれば、そのインスタンスは関数のように呼び出せます。

In [17]:
linear = nn.Linear(4, 5)
print(linear.__call__)

<bound method Module.__call__ of Linear(in_features=4, out_features=5, bias=True)>


上で定義したNetクラスを実際に使ってみます。

In [18]:
net = Net()  # ニューラルネットワークのインスタンスを生成

outputs = net(X_train[0:3])  # 訓練データの先頭から3個の要素を入力
print(outputs)
print(type(outputs))
print(outputs.dtype)
for idx in range(3):
    print('output:', outputs[idx].item(), ', label:', y_train[idx].item())

tensor([[-0.6376],
        [-0.6201],
        [-0.7957]], grad_fn=<AddmmBackward>)
<class 'torch.Tensor'>
torch.float32
output: -0.6375830769538879 , label: 1.0
output: -0.6200773119926453 , label: 2.0
output: -0.7957167029380798 , label: 0.0
